In [ ]:
# Code based on https://gist.github.com/Spaxe/94e130c73a1b835d3c30ea672ec7e5fe#file-csv-to-geojson-ipynb

In [1]:
import pandas
import json

In [2]:
import pandas as pd
import shutil, os, glob
import pymongo
from pymongo import MongoClient
import json
import schedule
import time
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [3]:
df = pd.concat(map(pd.read_csv, glob.glob('input' + '\*.csv')))
#Extract All source code
#https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

In [25]:
df.head()

,Date,Time,lat,lng,Type,Depth,Magnitude,ID,city,country
0,1965-01-02 00:00:00+00:00,13:44:18,19.246,145.616,Earthquake,131.6,6.0,ISCGEM860706,,
1,1965-01-04 00:00:00+00:00,11:29:49,1.863,127.352,Earthquake,80.0,5.8,ISCGEM860737,,
2,1965-01-05 00:00:00+00:00,18:05:58,-20.579,-173.972,Earthquake,20.0,6.2,ISCGEM860762,,
3,1965-01-08 00:00:00+00:00,18:49:43,-59.076,-23.557,Earthquake,15.0,5.8,ISCGEM860856,,
4,1965-01-09 00:00:00+00:00,13:32:50,11.938,126.427,Earthquake,15.0,5.8,ISCGEM860890,,


In [26]:
df.dtypes

Date          object
Time          object
lat          float64
lng          float64
Type          object
Depth        float64
Magnitude    float64
ID            object
city          object
country       object
dtype: object

In [27]:
# Before renaming the columns 
print(df.columns) 


Index(['Date', 'Time', 'lat', 'lng', 'Type', 'Depth', 'Magnitude', 'ID',
       'city', 'country'],
      dtype='object')


In [28]:

df.rename(columns = {'Latitude':'lat', 'Longitude':'lng'}, inplace = True) 
   
# After renaming the columns 
print(df.columns) 

Index(['Date', 'Time', 'lat', 'lng', 'Type', 'Depth', 'Magnitude', 'ID',
       'city', 'country'],
      dtype='object')


In [30]:
df.head()

,Date,Time,lat,lng,Type,Depth,Magnitude,ID,city,country
0,1965-01-02 00:00:00+00:00,13:44:18,19.246,145.616,Earthquake,131.6,6.0,ISCGEM860706,,
1,1965-01-04 00:00:00+00:00,11:29:49,1.863,127.352,Earthquake,80.0,5.8,ISCGEM860737,,
2,1965-01-05 00:00:00+00:00,18:05:58,-20.579,-173.972,Earthquake,20.0,6.2,ISCGEM860762,,
3,1965-01-08 00:00:00+00:00,18:49:43,-59.076,-23.557,Earthquake,15.0,5.8,ISCGEM860856,,
4,1965-01-09 00:00:00+00:00,13:32:50,11.938,126.427,Earthquake,15.0,5.8,ISCGEM860890,,


In [22]:
earthquakes = df

In [31]:
earthquakes.head()

,Date,Time,lat,lng,Type,Depth,Magnitude,ID,city,country
0,1965-01-02 00:00:00+00:00,13:44:18,19.246,145.616,Earthquake,131.6,6.0,ISCGEM860706,,
1,1965-01-04 00:00:00+00:00,11:29:49,1.863,127.352,Earthquake,80.0,5.8,ISCGEM860737,,
2,1965-01-05 00:00:00+00:00,18:05:58,-20.579,-173.972,Earthquake,20.0,6.2,ISCGEM860762,,
3,1965-01-08 00:00:00+00:00,18:49:43,-59.076,-23.557,Earthquake,15.0,5.8,ISCGEM860856,,
4,1965-01-09 00:00:00+00:00,13:32:50,11.938,126.427,Earthquake,15.0,5.8,ISCGEM860890,,


In [32]:
earthquakes['city'] = ""
earthquakes['country'] = ""
count = 0
for index, row in earthquakes.iterrows():
   near_city = citipy.nearest_city(row['lat'], row['lng']).city_name
   near_country = citipy.nearest_city(row['lat'], row['lng']).country_code
   earthquakes.set_value(index,"city",near_city)
   earthquakes.set_value(index,"country",near_country)
earthquakes.head()

C:\Users\mwals\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
C:\Users\mwals\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


,Date,Time,lat,lng,Type,Depth,Magnitude,ID,city,country
0,1965-01-02 00:00:00+00:00,13:44:18,19.246,145.616,Earthquake,131.6,6.0,ISCGEM860706,airai,pw
1,1965-01-04 00:00:00+00:00,11:29:49,1.863,127.352,Earthquake,80.0,5.8,ISCGEM860737,ternate,id
2,1965-01-05 00:00:00+00:00,18:05:58,-20.579,-173.972,Earthquake,20.0,6.2,ISCGEM860762,pangai,to
3,1965-01-08 00:00:00+00:00,18:49:43,-59.076,-23.557,Earthquake,15.0,5.8,ISCGEM860856,chuy,uy
4,1965-01-09 00:00:00+00:00,13:32:50,11.938,126.427,Earthquake,15.0,5.8,ISCGEM860890,dapdap,ph


In [33]:
json_result_string = df.to_json(
    orient='records', 
    double_precision=12,
    date_format='iso'
)
json_result = json.loads(json_result_string)

In [34]:
geojson = {
    'type': 'FeatureCollection',
    'features': []
}
for record in json_result:
    geojson['features'].append({
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [record['lng'], record['lat']],
        },
        'properties': record,
    })

In [35]:
# Writing converted data back to input folder
with open('input/quake_geoJSON_fmt.json', 'w') as f:
    f.write(json.dumps(geojson, indent=2))